In [9]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [10]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [11]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [12]:
#W = 250
hours = 24

insample_scenarios = Dict()
out_of_sample_scenarios = Dict()
seeds = [123,2,3,4,5,6,7,8,9,10,11,12,13,14,15]#
n_seeds = length(seeds)
Ws = [50, 100, 150, 200, 250, 600]


W_index = 0
for W in Ws

    for seed in seeds
        Random.seed!(seed)
        selected_scenarios = rand(1:n_scenarios, W)
        scenarios = Dict()
        counter = 1
        for i in selected_scenarios
            scenarios[counter] = all_scenarios[i]
            counter += 1
        end
        insample_scenarios[seed + n_seeds * W_index] = scenarios

        # create the list of index of scenarios out of sample
        out_of_sample = []
        for i in 1:n_scenarios
            if i ∉ selected_scenarios
                push!(out_of_sample, i)
            end
        end

        unselected_scenarios = Dict()
        counter = 1
        for i in out_of_sample
            unselected_scenarios[counter] = all_scenarios[i]
            counter += 1
        end
        out_of_sample_scenarios[seed + n_seeds * W_index] = unselected_scenarios
    end
    W_index += 1
end

alpha = 0.9
beta = 0.1
objectiv_values = []
exp_profits = []
profits_in = []
CVaR_values = []


Any[]

In [13]:
expected_profit_list = []
#create df with seeds as column and add the p_DA_values for each seed and definec the columns
bids_df = DataFrame()
W_index = 0

for W in Ws
    for seed in seeds
        scenarios = insample_scenarios[seed + n_seeds * (W_index)]

        # Create a new model with GLPK solver
        model = Model(GLPK.Optimizer)
        unregister(model, :p_DA)

        # Define the decision variables for hour
        @variable(model, p_DA[1:hours])
        @variable(model, delta[1:W,1:hours])
        @variable(model, delta_up[1:W,1:hours])
        @variable(model, delta_down[1:W,1:hours])
        @variable(model, zeta)
        @variable(model, eta[1:W] >= 0)

        # Define the objective function
        @objective(model, Max, (1-beta) *  sum(1/W*(scenarios[i][hour,"price"] * p_DA[hour]
            + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
            - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
            + beta  * (zeta - (1/(1-alpha)) * sum(1/W * eta[i] for i in 1:W))) for i in 1:W, hour in 1:hours))

        # Define the constraints
        @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
        @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
        @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
        @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
        @constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
        #@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
        @constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
        #@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)

        @constraint(model, [i in 1:W], -1 * sum(scenarios[i][hour,"price"] * p_DA[hour]
        + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2) for hour in 1:hours) + zeta - eta[i] <= 0)



        # Solve the optimization problem
        optimize!(model)

        # save the p_DA values
        p_DA_values = value.(p_DA)

        exp_profit = value.(sum(1/W.*(scenarios[i][hour,"price"] * p_DA[hour]
            + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
            - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2) for i in 1:W, hour in 1:hours)))


        #println(model)

        # Print the termination status
        status = termination_status(model)
        if status == MOI.OPTIMAL
            println("Expected profit ", exp_profit)
            println("p_DA: ", value.(p_DA))
            #add the p_DA values to the df
            bids_df[!, string(seed +  n_seeds * (W_index))] = value.(p_DA)
            push!(expected_profit_list, exp_profit)
            
        else
            println("No optimal solution found")
        end
    end
    W_index += 1
end

Expected profit 140082.63963812203
p_DA: [66.63129032258065, 14.389693451612905, 11.485499983870966, 9.385548338709675, 8.193596774193548, 45.95790322580645, 145.9269354516129, 7.711258064516128, 7.9080806451612915, 146.94499998387096, 9.519177403225806, 0.7419306451612897, 142.91516125806453, 57.317258, 135.9887096612903, 24.266177370967743, 146.0248386935484, 149.60145158064518, 10.317629032258065, 152.74596772580645, 4.493516129032258, 14.784612903225804, 6.808774177419355, 149.57870966129033]
Expected profit 105378.87847191561
p_DA: [22.33724183870968, 14.389693451612905, 15.94993545161286, 13.469580564516129, 9.242903161290323, 5.90827414516129, 145.9269354516129, 2.7339112741935483, 1.1038967741935486, 146.94499998387096, 0.7919354838709687, 0.7419306451612904, 142.91516125806453, 1.167741935483871, 135.9887096612903, 2.3072451612903224, 146.0248386935484, 149.60145158064518, 5.493548387096774, 152.74596772580645, 4.493516129032258, 4.803145161290321, 6.808774177419355, 149.57870

In [14]:
hours = 24
profits_list = []
W_index=0

for W in Ws
    for seed in seeds
        unselected_scenarios = out_of_sample_scenarios[seed +  n_seeds * (W_index)]
        W_new = length(unselected_scenarios)
        p_DA_values = value.(bids_df[!, string(seed +  n_seeds * (W_index))])
        # calculate the expected profit for the out of sample scenarios
        exp_profit = 0
        for i in 1:W_new
            profit_day = 0
            for hour in 1:hours
                profit = 0
                profit += unselected_scenarios[i][hour,"price"] * p_DA_values[hour] # should the day ahead price be fixed to the first scenario? So instead of i, 1?
                if unselected_scenarios[i][hour,"wind power"] > p_DA_values[hour]
                    profit += unselected_scenarios[i][hour,"price"] * (unselected_scenarios[i][hour,"wind power"] - p_DA_values[hour]) * (unselected_scenarios[i][hour,"grid_excess"]*0.9 + (1-unselected_scenarios[i][hour,"grid_excess"])*1)
                else
                    profit -= unselected_scenarios[i][hour,"price"] * (p_DA_values[hour] - unselected_scenarios[i][hour,"wind power"]) * (unselected_scenarios[i][hour,"grid_excess"]*1 + (1-unselected_scenarios[i][hour,"grid_excess"])*1.2)
                end
                profit_day += profit
            end
            #push!(profits_out, profit_day)
            exp_profit += profit_day
        end
        exp_profit = exp_profit / W_new
        push!(profits_list, exp_profit)
    end
    W_index += 1
end

profits_list

90-element Vector{Any}:
 130086.9417383111
 131242.7316639882
 130935.30425226795
 130981.61077546998
 130659.46961015632
 130823.06464843285
 130735.37410230684
 130743.6958033622
 131500.7942661526
 130093.6854216672
      ⋮
 130116.39777851515
 129449.23668153101
 131876.95928114626
 131969.6329476764
 132985.41172134032
 127106.1242486081
 132415.447654302
 131861.09964000247
 134654.71698812422

In [15]:
# create a dataframe with the expected profits, the seeds and the Ws
profits_df = DataFrame()

profits_df[!, "In sample scenarios"] = repeat(Ws, inner = length(seeds))
profits_df[!, "Seed"] = repeat(seeds, outer = length(Ws))
profits_df[!, "In sample profit"] = expected_profit_list
profits_df[!, "Out of sample profit"] = profits_list

# make a relative column
profits_df[!, "Relative profit"] = profits_df[!,"In sample profit"] ./ profits_df[!,"Out of sample profit"] .-1
profits_df

Row,In sample scenarios,Seed,In sample profit,Out of sample profit,Relative profit
,Int64,Int64,Any,Any,Float64
1,50,123,1.40083e5,1.30087e5,0.0768386
2,50,2,1.05379e5,1.31243e5,-0.197069
3,50,3,1.24294e5,1.30935e5,-0.0507238
4,50,4,1.21583e5,1.30982e5,-0.0717567
5,50,5,1.21309e5,1.30659e5,-0.0715651
6,50,6,1.28334e5,1.30823e5,-0.0190299
7,50,7,1.19664e5,1.30735e5,-0.0846858
8,50,8,1.28176e5,1.30744e5,-0.0196403
9,50,9,1.10376e5,1.31501e5,-0.160643


In [17]:
# save to a csv file 
CSV.write("1_5_results/relative_profits_two_price.csv", profits_df)

"1_5_results/relative_profits_two_price.csv"